In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import src.model as modelfunc
import src.sequencing as sequencing
from config import config

In [ ]:
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)  # Allow dynamic memory allocation
#         print("GPU memory pre-allocated successfully")
#     except RuntimeError as e:
#         print(e)

print("TensorFlow Version:", tf.__version__)
print("Is TensorFlow built with CUDA?", tf.test.is_built_with_cuda())
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPU Details:", tf.config.list_physical_devices('GPU'))


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if tf.config.list_physical_devices('GPU'):
    print("Running on GPU")
else:
    print("GPU not available. Running on CPU")

## parse GPMF

In [ ]:
df = pd.read_csv(config.DATA_DIR / "test_imu_frameindex_label_data.csv")  # Replace with your data file
overlap = 0.0
length = 10
num_actions = 5  # Example number of actions
df.dropna(inplace=True)

display(df)

# Convert LABEL to categorical and save mappings
df["LABEL"] = df["LABEL"].astype("category")
label_mapping = dict(enumerate(df["LABEL"].cat.categories))
df["LABEL"] = df["LABEL"].cat.codes

display(df)
print(label_mapping)

## Load data

In [ ]:
# Preprocess data
# padded_sequences, padded_labels = sequencing.get_sequences_and_labels(df, overlap, length, num_actions)
sequences = sequencing.create_sequence(df, overlap, length)
padded_sequences = sequencing.get_sequences_pure_data(sequences)
padded_labels = sequencing.get_pure_labels(sequences)
padded_labels = tf.one_hot(padded_labels, depth=5)

# # Build and train model
# input_shape = (X_train.shape[1], X_train.shape[2])
# model = build_seq2seq_lstm(input_shape, num_classes=num_actions)
# history = train_model(model, X_train, y_train)

In [5]:
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Model parameters
# timesteps = None # Variable-length sequences
# features = 6  # IMU features (e.g., ax, ay, az, gx, gy, gz)
# num_classes = 5  # Actions (e.g., left turn, right turn, stopping)

# # Simulated variable-length IMU data
# sequences = [np.random.rand(np.random.randint(1000, 2000), features) for _ in range(100)]
# labels = [np.random.randint(0, num_classes, len(seq)) for seq in sequences]  # One label per timestep
# print(type(sequences))
# # Pad sequences
# padded_sequences = pad_sequences(sequences, padding='post', dtype='float32')
# padded_labels = pad_sequences(labels, padding='post', dtype='int32')

# # One-hot encode labels
# padded_labels = tf.keras.utils.to_categorical(padded_labels, num_classes)

# padded_sequences = padded_sequences.astype('float32')
# padded_labels = padded_labels.astype('float32')

# # Check shapes
# print("Padded sequences shape:", padded_sequences.shape)
# print("Padded labels shape:", padded_labels.shape)
# print("Padded sequences dtype:", padded_sequences.dtype)  # Should be float32
# print("Padded labels dtype:", padded_labels.dtype)        # Should be float32


In [ ]:
# Split chronologically
train_size = int(0.5 * len(padded_sequences))  # 80% for training, 20% for validation
X_train, X_val = padded_sequences[:train_size], padded_sequences[train_size:]
y_train, y_val = padded_labels[:train_size], padded_labels[train_size:]

print("Train shape:", X_train.shape, y_train.shape)
print("Validation shape:", X_val.shape, y_val.shape)

In [6]:
# Generate sample weights to ignore padded timesteps
sample_weights = np.array([
    [1 if np.any(timestep != 0) else 0 for timestep in sequence]
    for sequence in X_train
])

## Build Model

In [ ]:
# Model parameters
print(padded_sequences.shape)
timesteps = padded_sequences.shape[1] # Variable-length sequences
features = 6  # IMU features (e.g., ax, ay, az, gx, gy, gz)
num_classes = 5  # Actions (e.g., left turn, right turn, stopping)
print(timesteps)
# Build the model
model = modelfunc.build_seq2seq_lstm((timesteps, features), num_classes, dropout=0.4)
model.summary()

## Train Model

In [ ]:
# _ = model.predict(X_train[:1])  # Perform a forward pass to initialize the model

# Build the model explicitly
model.build(input_shape=(None, timesteps, 6))  # Example: batch size = None, 50 timesteps, 6 features
model.make_train_function()
model.make_predict_function()

In [ ]:
# # Example call to the function
# history = modelfunc.train_model(
#     model=model,  # Your pre-defined Keras model
#     X_train=padded_sequences, 
#     y_train=padded_labels,
#     sample_weight=sample_weights,  # Optional; pass None if not using sample weights
#     batch_size=16,  # Optional; defaults to 16
#     epochs=50       # Optional; defaults to 10
# )

# Example call to the function
history = modelfunc.train_model(
    model=model,  # Your pre-defined Keras model
    X_train=X_train, 
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    sample_weight=sample_weights,  # Optional; pass None if not using sample weights
    batch_size=16,  # Optional; defaults to 16
    epochs=10       # Optional; defaults to 10
)

## Model prediction

In [ ]:
# # Simulated test data
# test_sequence = np.random.rand(80, features)  # Variable-length sequence
# test_sequence_padded = pad_sequences([test_sequence], maxlen=padded_sequences.shape[1], padding='post')

# Make predictions
predictions = model.predict(padded_sequences)

# Get the confidence scores and the predicted classes
confidence_scores = np.max(predictions, axis=-1)  # Maximum probability for each timestep
predicted_classes = np.argmax(predictions, axis=-1)  # Class with the highest probability for each timestep

# Print results
print("Confidence scores shape:", confidence_scores.shape)  # Same shape as the input sequence
print("Predicted classes shape:", predicted_classes.shape)
print(predicted_classes)
# # Example: For a specific sequence
# example_idx = 0  # Index of the sequence to inspect
# print(f"Sequence {example_idx}:")
# print("Predicted classes:", predicted_classes[example_idx])
# print("Confidence scores:", confidence_scores[example_idx])

In [ ]:
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=10,
        batch_size=16
    )